## Imports

In [40]:
import cv2
import os
import numpy as np
from tqdm import tqdm
from pathlib import Path
from matplotlib import pyplot as plt

## Settings

In [41]:
base_dir = '/home/drevital/obstacles_classification_datasets/black_belt'
in_dir = os.path.join(base_dir, 'no_mask')
threshold = 55
belt_sizes = [0, 10, 20, 30, 40, 50]

## Build Belt & Mask & Compare to No Belt Masks

In [42]:
imnames = os.listdir(in_dir)

for imname in tqdm(imnames):
    for belt_size in belt_sizes:
        impath = os.path.join(in_dir, imname)
        no_mask = cv2.imread(impath)

        bg_h = no_mask.shape[0]
        bg_w = no_mask.shape[1]//2
        bg = np.zeros((bg_h + belt_size*2, bg_w + belt_size*2, 3), np.uint8)

        w = no_mask.shape[1]
        ref = no_mask[:, :w//2]
        current = no_mask[:, w//2:]
        xref = np.copy(bg)
        xref[belt_size:belt_size+bg_h, belt_size:belt_size+bg_w] = ref
        xcurrent = np.copy(bg)
        xcurrent[belt_size:belt_size+bg_h, belt_size:belt_size+bg_w] = current

        # Add ref and current on top of the black background, so they have a black belt at required size

        diff = cv2.absdiff(xcurrent, xref)
        agg_rgb = np.stack((diff[:, :, 0], diff[:, :, 1], diff[:, :, 2])).max(0)
        _, mask = cv2.threshold(agg_rgb, threshold, 255, cv2.THRESH_BINARY)

        # old morphological operations
        copyImg = cv2.erode(mask, np.ones((3, 3), np.uint8), iterations=1)  # reduce noise
        copyImg = cv2.dilate(copyImg, np.ones((7, 7), np.uint8), iterations=1)
        copyImg = cv2.erode(copyImg, np.ones((5, 5), np.uint8), iterations=1)
        copyImg = cv2.dilate(copyImg, np.ones((9, 9), np.uint8), iterations=1)
        kernel = np.ones((11, 11), np.uint8)  # kernel for dilation

        # increase area to an object
        copyImg = cv2.dilate(copyImg, kernel, iterations=2)
        copyImg = cv2.dilate(copyImg, np.ones((13, 13), np.uint8), iterations=1)
        copyImg = cv2.erode(copyImg, np.ones((11, 11), np.uint8), iterations=1)
        copyImg = cv2.erode(copyImg, np.ones((5, 5), np.uint8), iterations=1)

        # Crop mask image to actual size
        mask = copyImg 
        mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
        mask = mask[belt_size:belt_size+bg_h, belt_size:belt_size+bg_w]

        # Generate concatenated image (ref, current, mask)
        concat = cv2.hconcat([ref, current, mask])

        # Store image
        out_dir = os.path.join(base_dir, str(belt_size))
        Path(out_dir).mkdir(parents=True, exist_ok=True)
        out_path = os.path.join(out_dir, imname)
        cv2.imwrite(out_path, concat)    

100%|██████████| 100/100 [00:02<00:00, 39.87it/s]


## Generate Comparison Parameters

In [43]:
any_diffs = {}
white_diffs = {}

for imname in imnames[:1]:
    for belt_size in belt_sizes:
        inference_path = os.path.join(base_dir, 'inference', imname)
        inference = cv2.imread(inference_path)
        w = inference.shape[1]
        inference_mask = inference[:, (w*2)//3:]/255.
        inference_mask = np.array(inference_mask, dtype=np.uint8)
        inference_mask = cv2.cvtColor(inference_mask, cv2.COLOR_RGB2GRAY)
        belt_path = os.path.join(base_dir, str(belt_size), imname)
        belt = cv2.imread(belt_path)
        w = belt.shape[1]
        belt_mask = belt[:, (w*2)//3:]/255.
        belt_mask = np.array(belt_mask, dtype=np.uint8)
        belt_mask = cv2.cvtColor(belt_mask, cv2.COLOR_RGB2GRAY)
        any_diffs[belt_size] = np.sum(inference_mask ^ belt_mask)
        white_diffs[belt_size] = np.sum(inference_mask) - np.sum(belt_mask)

In [44]:
any_diffs

{0: 22323, 10: 22638, 20: 22638, 30: 22638, 40: 22638, 50: 22638}

In [45]:
white_diffs

{0: 22323, 10: 22638, 20: 22638, 30: 22638, 40: 22638, 50: 22638}